# 141B Final Project: Vaccine Hesitancy in the United States
## by Antonio Pelayo, Gianni Spiga, and Sharon Vien

In this notebook, we explore data provided by the CDC in regards to the hesistancy of adults of different races in the United States in hopes to find any patterns or trends that might help better prepare for more efficient vaccine rollout on a nationwide and statewide scale in the future.

### Extracting the Data

### Addressing and Understanding our Variables

### Visualizing the Data